# UGCrawler 1.0 alpha

This Notebook contains code fragments to crawl ultimate-guitar.com. It is far from finished but all the major components are there. 

Since it is not feasible for me to generate a complete rip, i am sharing this with all the guitarists out there who are pissed off by UG for any of the million reasons given. 

If you would like to contribute, by doing part of the crawl, of implementin a better selection logic, or in any other way, feel free to do so.

In [ ]:
from lxml import etree
import lxml.html
import string
import urllib.request
from time import sleep
import sys
import random

In [ ]:
chars = ['0-9'] + list(string.ascii_lowercase)

In [ ]:
def get_band_urls(char):    
    base_url = 'https://www.ultimate-guitar.com/bands/{}.htm'
    
    htmlparser = etree.HTMLParser()
    
    band_dict = {}
    
    cnt = 0
    while True:
        sleep(random.uniform(.8, 1.2))
        
        postfix = '' if cnt == 0 else cnt
        response = urllib.request.urlopen(base_url.format(char + str(postfix)))
        tree = etree.parse(response, htmlparser)

        # collect song links
        tab_table = tree.xpath('//*[contains(@class, "b3")]/../table[2]/tr/td[2]/a')
        
        # empty means no more pages
        if not tab_table:
            break
            
        for child in tab_table:
            band_dict[child.text[:-5]] = child.attrib['href']
        cnt += 1
        
    return band_dict

In [ ]:
def get_artist_links(url):
    base_url = 'https://www.ultimate-guitar.com'

    url = base_url + url[:-4] + '{}' + url[-4:]
    
    htmlparser = etree.HTMLParser()
    
    all_tabs_chords = []
    
    cnt = 0
    tries = 0
    while True:
        sleep(random.uniform(.8, 1.2))
        
        # page 0 exists and is pretty much page 1 without the album shit
        if cnt == 1:
            cnt = 2
            continue
            
        try:
            response = urllib.request.urlopen(url.format(cnt))
        except urllib.error.HTTPError as e:
            if e.code == 404:
                # end of tab sites
                break
            else:
                # retry
                if tries > 5:
                    cnt += 1
                    tries = 0
                else:
                    tries += 1
                continue
                    
        tries = 0
        
        tree = etree.parse(response, htmlparser)
        
        # collect entries
        tab_table = tree.xpath("//td/b[./text()='Tab' or ./text()='tab']/../..")
        chords_table = tree.xpath("//td/b[./text()='Chords' or ./text()='chords']/../..")

        for song in tab_table:
            name = song.xpath('./td[1]/a[1]')[0].text    
            link = song.xpath('./td[1]/a[1]')[0].attrib['href']

            if not song.xpath('./td[2]/span[1]'):
                rating = 0
            else:
                rating = song.xpath('./td[2]/span[1]')[0].attrib['title']

            all_tabs_chords.append(
                {
                    'name': name,
                    'link': link,
                    'rating': float(rating),
                    'type': 'tab'
                }
            )
            
        for song in chords_table:
            name = song.xpath('./td[1]/a[1]')[0].text    
            link = song.xpath('./td[1]/a[1]')[0].attrib['href']

            if not song.xpath('./td[2]/span[1]'):
                rating = 0
            else:
                rating = song.xpath('./td[2]/span[1]')[0].attrib['title']

            all_tabs_chords.append(
                {
                    'name': name,
                    'link': link,
                    'rating': float(rating),
                    'type': 'chords'
                }
            )        
        cnt += 1
        
    return all_tabs_chords

In [ ]:
def get_print_version(url):
    tries = 0
    while tries < 5:
        try:
            response = urllib.request.urlopen(url)
        except urllib.error.HTTPError as e:
            if e.code == 404:
                return None
            else:
                tries += 1
                continue
                
        htmlparser = etree.HTMLParser()
        tree = etree.parse(response, htmlparser)
        
        # get link
        link = tree.xpath("//*[@id='print_link']")[0].attrib['href']
        response = urllib.request.urlopen('https://tabs.ultimate-guitar.com{}'.format(link))
        
        ht = lxml.html.parse(response)
        return ht.xpath("//div[contains(@class, 'tb_ct')]")[0].text_content()

### Here are some examples of how the functions work, and what they return, should be self explanatory

In [ ]:
band_urls = get_band_urls('w')
band_urls

In [ ]:
artist_links = get_artist_links('/tabs/we_are_scientists_tabs.htm')
artist_links

In [ ]:
tab = get_print_version('https://tabs.ultimate-guitar.com/tab/we_are_scientists/this_scene_is_dead_tabs_844257')
print(tab)

In [ ]:
def clean(s):
    return ' '.join(re.sub('[^a-zA-Z0-9\(\)\-\s"\'!?#$\+]', '', s).split())

# UNLEASH THE SPIDER

In [ ]:
import re, os

highest_rated_only = True
target_dir = '/home/max/ssd_2/UG/'

for c in chars:
    # rip the band urls
    band_urls = get_band_urls(c)
    
    # rip the tabs for each band
    for artist, url in band_urls.items():
        
        # make out dirs
        artist_folder_name = clean(artist)
        artist_dir = os.path.join(target_dir, c, artist_folder_name)
        
        if not os.path.exists(artist_dir):
            os.makedirs(artist_dir)
            os.makedirs(os.path.join(artist_dir, 'chords'))
            os.makedirs(os.path.join(artist_dir, 'tab'))
        else:
            # assume we finished crawling this artist
            continue
        
        artist_dict = {}
        artist_dict['chords'] = {}
        artist_dict['tab'] = {}
        
        # rip
        print('Crawling {}...'.format(artist))
        song_links = get_artist_links(url)

        # keep the highest rated version, factoring in rating count        
        for song_dict in song_links:
            t = song_dict['type']
            
            if highest_rated_only:
                cleaned_name = clean(song_dict['name'])
                
                if cleaned_name not in artist_dict[t].keys() \
                or artist_dict[t][cleaned_name]['rating'] < song_dict['rating']:
                    artist_dict[t][cleaned_name] = song_dict
            else:
                artist_dict[t][song_dict['name']] = song_dict                
            
        # rip and write songs to drive
        for name, d in artist_dict['tab'].items():
            tab = get_print_version(d['link'])
            
            file_name = clean(name) + '.txt'
            
            with open(os.path.join(artist_dir, 'tab', file_name), 'w') as out:
                out.write(tab)
            
            
        for name, d in artist_dict['chords'].items():
            chords = get_print_version(d['link'])
            
            file_name = clean(name) + '.txt'
            
            with open(os.path.join(artist_dir, 'chords', file_name), 'w') as out:
                out.write(chords)

        

    sleep(5)

In [ ]:
random.uniform(.8, 1.2)